In [1]:
pip install pandas json time tqdm langchain_groq langchain_core dotenv


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
import pandas as pd # 데이터 프레임 처리
import json # 데이터 파일 읽기
import time # 시간 처리
from tqdm import tqdm # 진행 상황 표시
from langchain_groq import ChatGroq # 챗봇 모델 사용
from langchain_core.messages import HumanMessage, AIMessage # 메시지 형식
from dotenv import load_dotenv # 환경 변수 처리
import os # 환경 변수 처리


In [29]:
##1단계: CSV 파일 불러오기 및 기본 확인
# CSV 파일 로드
df = pd.read_csv('products_termdeposit.csv')

# 데이터프레임 기본 정보 확인
print("1. 데이터 크기:")
print(f"행 개수: {df.shape[0]}")
print(f"열 개수: {df.shape[1]}")

print("\n2. 컬럼명:")
print(df.columns.tolist())

print("\n3. 데이터 미리보기:")
print(df.head())

print("\n4. 데이터 타입:")
print(df.dtypes)


1. 데이터 크기:
행 개수: 242
열 개수: 11

2. 컬럼명:
['1', '정기 예금', 'WR0001B', '우리은행', 'WON플러스예금', '- 가입기간: 1~36개월\\\n- 최소가입금액: 1만원 이상\\\n- 만기일을 일,월 단위로 자유롭게 선택 가능\\\n- 만기해지 시 신규일 당시 영업점과 인터넷 홈페이지에 고시된 계약기간별 금리 적용', '1.1', '실명의 개인', '인터넷,스마트폰,전화(텔레뱅킹)', '해당사항 없음', '20241120']

3. 데이터 미리보기:
                             1  정기 예금            WR0001B         우리은행  \
0                            2  정기 예금           00320342  한국스탠다드차타드은행   
1                            3  정기 예금  10511008000996000        아이엠뱅크   
2                            4  정기 예금  10511008001004000        아이엠뱅크   
3                            5  정기 예금  10511008001166004        아이엠뱅크   
4  -당행 오픈뱅킹서비스에 다른 은행 계좌 등록시 \    NaN                NaN          NaN   

            WON플러스예금  \
0          e-그린세이브예금   
1  iM주거래우대예금(첫만남고객형)   
2     iM행복파트너예금(일반형)   
3             iM함께예금   
4                NaN   

  - 가입기간: 1~36개월\\n- 최소가입금액: 1만원 이상\\n- 만기일을 일,월 단위로 자유롭게 선택 가능\\n- 만기해지 시 신규일 당시 영업점과 인터넷 홈페이지에 고시된 계약기간별 금리 적용  \
0                      

In [30]:
#2단계 한글로 칼럼명 변경
# 현재 컬럼명을 보기 좋게 한글로 변경
column_names = ['인덱스', '상품구분', '상품코드', '은행명', '상품명', 
                '상품설명', '금리', '가입대상', '가입채널', '우대사항', '등록일자']

df.columns = column_names

print("\n5. 변경된 컬럼명 확인:")
print(df.columns.tolist())


5. 변경된 컬럼명 확인:
['인덱스', '상품구분', '상품코드', '은행명', '상품명', '상품설명', '금리', '가입대상', '가입채널', '우대사항', '등록일자']


In [34]:
# 결측치 확인
# 결측치 확인
print("\n6. 결측치 확인:")
print(df.isnull().sum())
#결측치 NULL 값 처리
df = df.fillna('NULL')
print(df.isnull().sum())

# 데이터 기본 통계
print("\n7. 수치형 데이터 기본 통계:")
print(df.describe())

# 각 컬럼의 유니크 값 개수 확인
print("\n8. 각 컬럼의 고유값 개수:")
print(df.nunique())


6. 결측치 확인:
인덱스     0
상품구분    0
상품코드    0
은행명     0
상품명     0
상품설명    0
금리      0
가입대상    0
가입채널    0
우대사항    0
등록일자    0
dtype: int64
인덱스     0
상품구분    0
상품코드    0
은행명     0
상품명     0
상품설명    0
금리      0
가입대상    0
가입채널    0
우대사항    0
등록일자    0
dtype: int64

7. 수치형 데이터 기본 통계:
        인덱스   상품구분  상품코드     은행명   상품명 상품설명     금리  가입대상      가입채널 우대사항  \
count   242    242   242     242   242  242  242.0   242       242  242   
unique  242      4   240      44   198  155    3.0    53        13  115   
top       2  정기 예금  NULL  다올저축은행  정기예금   없음    1.0  제한없음  인터넷,스마트폰   없음   
freq      1    122     3      13    13   27  231.0   104        62   82   

              등록일자  
count        242.0  
unique        40.0  
top     20241120.0  
freq          96.0  

8. 각 컬럼의 고유값 개수:
인덱스     242
상품구분      4
상품코드    240
은행명      44
상품명     198
상품설명    155
금리        3
가입대상     53
가입채널     13
우대사항    115
등록일자     40
dtype: int64


In [3]:

load_dotenv() # 환경 변수 로드
groq_api_key = os.getenv("GROQ_API_KEY") # 환경 변수에서 GROQ API 키 로드


In [ ]:
# CSV 파일 로드
df = pd.read_csv('products_termdeposit.csv')

# 데이터 크기 확인
print("행 개수:", df.shape[0])
print("열 개수:", df.shape[1])
print("컬럼명:", df.columns)

행 개수: 242
열 개수: 11
컬럼명: Index(['1', '정기 예금', 'WR0001B', '우리은행', 'WON플러스예금',
       '- 가입기간: 1~36개월\\n- 최소가입금액: 1만원 이상\\n- 만기일을 일,월 단위로 자유롭게 선택 가능\\n- 만기해지 시 신규일 당시 영업점과 인터넷 홈페이지에 고시된 계약기간별 금리 적용',
       '1.1', '실명의 개인', '인터넷,스마트폰,전화(텔레뱅킹)', '해당사항 없음', '20241120'],
      dtype='object')


#2. GROQ LLM 모델 설정

In [10]:
groq_llm = ChatGroq(
    model="llama-3.1-70b-versatile",    # 모델 설정
    temperature=0,                      # 일관된 응답을 위해 0으로 설정
    max_tokens=4096,                    # 최대 토큰 수 설정
    api_key=groq_api_key,                # API 키 설정
    timeout=10,                         # 요청 시간 초과 설정
    max_retries=3                        # 최대 재시도 횟수 설정
)

In [36]:
def process_product_description(text):
    prompt = f"""
    다음 금융상품 설명을 분석하여 아래 형식으로 정리해주세요:
    {text}

    형식:
    가입기간:
    최소가입금액:
    주요특징:
    """
    
    try:
        response = groq_llm.invoke([
            {"role": "user", "content": prompt}
        ])
        return response.content
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return text

def process_benefits(text):
    prompt = f"""
    다음 금융상품 우대사항을 분석하여 아래 형식으로 정리해주세요:
    {text}

    형식:
    우대조건:
    우대혜택:
    비고사항:
    """
    
    try:
        response = groq_llm.invoke([
            {"role": "user", "content": prompt}
        ])
        return response.content
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return text

In [37]:
from tqdm import tqdm
import time

# 처리된 결과를 저장할 새로운 데이터프레임 생성
processed_df = df.copy()

# 진행바와 함께 전체 데이터 처리
print("전체 데이터 처리 시작...")
for idx in tqdm(processed_df.index):
    # API 호출 제한을 고려한 딜레이 추가
    time.sleep(0.5)
    
    try:
        # 상품설명 처리
        processed_df.at[idx, '상품설명'] = process_product_description(
            processed_df.loc[idx, '상품설명']
        )
        
        # 우대사항 처리
        processed_df.at[idx, '우대사항'] = process_benefits(
            processed_df.loc[idx, '우대사항']
        )
        
    except Exception as e:
        print(f"\n행 {idx} 처리 중 오류 발생: {e}")
        continue

# 처리 결과 확인
print("\n처리 완료된 데이터 샘플:")
print(processed_df[['상품명', '상품설명', '우대사항']].head())

# 처리된 데이터의 기본 통계 확인
print("\n처리된 데이터 정보:")
print(processed_df.info())

전체 데이터 처리 시작...


100%|██████████| 242/242 [20:54<00:00,  5.18s/it]


처리 완료된 데이터 샘플:
                 상품명                                               상품설명  \
0          e-그린세이브예금  금융상품의 설명이 제공되지 않았습니다. 제공된 설명에 따라 다음과 같이 정리하였습니...   
1  iM주거래우대예금(첫만남고객형)  以下은 금융상품의 설명을 분석하여 정리한 결과입니다.\n\n* 가입기간: (없음)\...   
2     iM행복파트너예금(일반형)  금융상품 설명 분석 결과입니다.\n\n* 가입기간: 설명에서 언급되지 않았습니다.\...   
3             iM함께예금  금융상품의 주요 특징은 다음과 같습니다.\n\n* 가입기간: (없음)\n* 최소가입...   
4               NULL  제공된 정보가 없으므로, 빈칸으로 표시합니다.\n\n가입기간: \n최소가입금액: \...   

                                                우대사항  
0  다음은 금융상품 우대사항을 분석하여 정리한 결과입니다.\n\n**우대사항 1**\n...  
1  우대조건:\n1. 목돈굴리기예금 최초 가입\n2. 신용(체크)카드 신규 발급 (최근...  
2  우대조건:\n1. 지난달 당행 통장으로 연금 입금 실적 보유\n2. 상품 가입 전 ...  
3  우대조건: 전월 총 수신 평잔실적 또는 상품 가입 전 첫만남플러스 통장 보유, 당행...  
4  제공된 정보가 없습니다. 금융상품 우대사항을 분석하여 정리해드리겠습니다.\n\nNU...  

처리된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   인덱스     

In [ ]:
# 처리 전후 비교
def compare_changes(original_df, processed_df, column):
    print(f"\n{column} 변경 사례:")
    for idx in range(3):  # 처음 3개 행만 비교
        print(f"\n행 {idx}:")
        print("처리 전:", original_df.loc[idx, column])
        print("처리 후:", processed_df.loc[idx, column])
        print("-" * 50)

# 상품설명과 우대사항 컬럼의 변경사항 확인
compare_changes(df, processed_df, '상품설명')
compare_changes(df, processed_df, '우대사항')

# 처리 실패 여부 확인
def check_processing_failures(original_df, processed_df):
    failures = 0
    for idx in processed_df.index:
        if (processed_df.loc[idx, '상품설명'] == original_df.loc[idx, '상품설명'] and 
            processed_df.loc[idx, '우대사항'] == original_df.loc[idx, '우대사항']):
            failures += 1
    
    return failures

failures = check_processing_failures(df, processed_df)
print(f"\n처리 실패 건수: {failures}")